In [1]:
import numpy as np
import plotly.graph_objects as go
import meshio

In [2]:
import plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from numpy import pi, cos, sin, sqrt
import meshio
import math
import time as t

In [3]:
# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("x1 graph", "x2 graph", "y1 graph", "y2 graph")
)

In [4]:
#data frame from rounded data file
update_file_name = 'angles_steady.csv'
df = pd.read_csv(update_file_name)
rounded = np.round(df)

#find optimal and delete it from data frame
optimal = df.tail(1)
x1_optimal = optimal['x1'].tolist()[0]
y1_optimal = optimal['y1'].tolist()[0]
# # print(x_optimal, y_optimal)
print(optimal)
x2_optimal = optimal['x2'].tolist()[0]
y2_optimal = optimal['y2'].tolist()[0]
df = df.head(-1)

#find all par for graphs
time = df['computer_time'].tolist()
start_time = time[0]
time = [i-start_time for i in time]
x1 = df['x1'].tolist()
x1 = [i+x1_optimal for i in x1]
y1 = df['y1'].tolist()
y1 = [i-y1_optimal for i in y1]
# z1 = df['z1'].tolist()

x2 = df['x2'].tolist()
x2 = [i+x2_optimal for i in x2]
y2 = df['y2'].tolist()
y2 = [i-y2_optimal for i in y2]
# z2 = df['z2'].tolist()


    human_time  computer_time        x1         y1        z1        x2  \
244          0            0.0  1.541047 -11.415431  0.818528 -0.866363   

            y2        z2  
244  13.812965 -0.871074  


In [5]:
# Add traces
fig.add_trace(go.Scatter(x=time, y=x1), row=1, col=1)
fig.add_trace(go.Scatter(x=time, y=y1), row=2, col=1)
# fig.add_trace(go.Scatter(x=time, y=z1), row=3, col=1)
fig.add_trace(go.Scatter(x=time, y=x2), row=1, col=2)
fig.add_trace(go.Scatter(x=time, y=y2), row=2, col=2)
# fig.add_trace(go.Scatter(x=time, y=z2), row=3, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="time", row=1, col=1)
fig.update_xaxes(title_text="time", row=2, col=1)
# fig.update_xaxes(title_text="time", row=3, col=1)
fig.update_xaxes(title_text="time", row=1, col=2)
fig.update_xaxes(title_text="time", row=2, col=2)
# fig.update_xaxes(title_text="time", row=3, col=2)

# # Update yaxis properties
fig.update_yaxes(title_text="x1", row=1, col=1, range=[-90, 90])
fig.update_yaxes(title_text="y1", row=2, col=1, range=[-90, 90])
# fig.update_yaxes(title_text="z1", row=3, col=1)
fig.update_yaxes(title_text="x2", row=1, col=2, range=[-90, 90])
fig.update_yaxes(title_text="y2", row=2, col=2, range=[-90, 90])
# fig.update_yaxes(title_text="z2", row=3, col=2)

# Update title and height
fig.update_layout(title_text="Posture position", height=650)

fig.write_html('first_figure.html', auto_open=True)

In [6]:
from numpy import pi, cos, sin, sqrt

def rotx(t):
    return np.array([[1, 0, 0], [0, cos(t), -sin(t)], [0, sin(t), cos(t)]])

def roty(t):
    return np.array([[cos(t), 0, sin(t)],  [0, 1, 0],[-sin(t), 0, cos(t)]])   
    
def rotz(t):
    return np.array([[cos(t), -sin(t), 0], [sin(t), cos(t), 0], [0, 0, 1]])

def rotxyz(z, y, x):
    return np.array([[cos(y)*cos(z), -cos(x)*sin(z)+sin(x)*sin(y)*cos(z), sin(x)*sin(z)+cos(x)*sin(y)*cos(z)],
                    [cos(y)*sin(z), cos(x)*sin(z)+sin(x)*sin(y)*sin(z), -sin(x)*cos(z)+cos(x)*sin(y)*sin(z)],
                    [-sin(y), sin(x)*cos(y), cos(x)*cos(y)]])

In [7]:
mesh = meshio.read('airplane.ply', file_format='ply')

In [8]:
points = rotz(0).dot(mesh.points.T)
points.shape

(3, 1335)

In [9]:
D = np.array([-1283.45876179,  -431.01578484,    63])
x0, y0, z0 = D
u1 = np.array([sqrt(3)/2, -0.5, 0 ])
u2 = np.array([0.5, sqrt(3)/2, 0])
u3 = np.cross(u1, u2)
A = np.array([u1,u2,u3]) #T_{B'B}   B'=(u1, u2, u3), A is the matrix of transition from 
                         #the orthonormal basis, B', to the standard basis in the 3d space
A

array([[ 0.8660254, -0.5      ,  0.       ],
       [ 0.5      ,  0.8660254,  0.       ],
       [-0.       ,  0.       ,  1.       ]])

In [10]:
def rotate(points, D, A, phi, rot=rotx):
    #points - array of shape (n, 3); they give the initial position of the airplane points
    #D - aray of shape (3,), the origin of the linear frae tied to the airplane; D was defined aboeve
    # A - array of shape (3, 3);  A = np.array([u1,u2,u3])
    # phi - the angle of rotation in radians
    # rot - one of the functions rotx, roty, rotz that returns  the rotation matrix of angle phi;
    # rotx, roty, roz define respectively: the roll, pitch and yaw motion
    
    points_n = np.dot(A, (points-D).T) # express the airplane points with respect to the linear frame (D; u1, u2, u3)
    p_rotated = rot(phi).dot(points_n) #airplane points rotated and expressed with respect to (D; u1, u2, u3)
    return ((A.T).dot(p_rotated)).T  + D   # returns rotated points of shape (n, 3)) and of coords 
                                           # with respect to the standard linear frame (O; i, j, k)
   

In [11]:
x, y, z = points
I, J, K = mesh.cells[0].data.T

In [12]:
gr= [[0, 'rgb(150, 150, 150)'], 
     [1, 'rgb(150, 150, 150)']]

In [13]:
fig = go.Figure(go.Mesh3d(
            x=x,
            y=y, 
            z=z, 
            i=I, 
            j=J, 
            k=K, 
            colorscale=gr,
            showscale=False,
            intensity=z,
            lighting=dict(ambient=0.5,
                          diffuse=1,
                          fresnel=4,        
                          specular=0.5,
                          roughness=0.5),
            lightposition=dict(x=100,
                               y=100,
                               z=1000
            )))


fig.update_layout(title_text='', title_x=0.5, title_y=0.8, 
                  width=800, height=800,scene_aspectmode='data', 
                  scene_xaxis_visible=False,  scene_yaxis_visible=False,
                  scene_zaxis_visible=False,
                  scene_camera_eye=dict(x=1.35,y=1.35, z=0.5));

In [14]:
# ty = np.linspace(0, 2*pi/9, 24) #ty[k] successive angles of yaw rotation
# print(ty)
# # tr = np.linspace(0, 7*pi/36, 20) # tr[k] successive angles of pitch rotation
tr = y1

print(x1)
# x_angle = [math.radians(i) for i in y1]
# y_angle = [math.radians(i) for i in x1]
x_angle = y1
y_angle = x1

frames=[]
# for k in range(len(ty)):
#     x, y, z = rotate(points.T, D, A,  ty[k], rot=rotz).T
#     frames.append(go.Frame(data=[go.Mesh3d(x=x,y=y,z=z)],
#                            layout=dict(title_text='Yaw motion'),
#                            traces=[0]))

for k in range(len(tr)): 
    x, y, z = rotate(points.T, D, A,  tr[k], rot=rotx).T 
    frames.append(go.Frame(data=[go.Mesh3d(x=x,y=y,z=z)],
                           layout=dict(title_text='Pitch motion'),
                           traces=[0]))

# for i in range(len(x_angle)): 
#     x, y, z = rotate(points.T, D, A, 0, y_angle[i], x_angle[i], rot=rotxyz).T 
#     frames.append(go.Frame(data=[go.Mesh3d(x=x,y=y,z=z)],
#                            layout=dict(title_text='Pitch motion'),
#                            traces=[0]))
# print(x_angle)
# print(y_angle)
# for i in range(len(x_angle)): 
#     x, y, z = rotate(points.T, D, A, x_angle[i], rot=rotx).T 
#     frames.append(go.Frame(data=[go.Mesh3d(x=x,y=y,z=z)],
#                            layout=dict(title_text='Pitch motion'),
#                            traces=[0]))

[-0.17966286132266496, -0.17926772775513444, -0.0011067715278416657, 0.136074719509665, 0.2529048492154098, 0.3317389000936135, 0.2627722177352201, 0.26896483848896824, -0.02006356507342888, -0.1481318232528035, -0.2742078013911442, -0.1421655418305292, -0.10141027459876728, -0.2216889537221991, -0.5644787329083161, -1.317601107803256, -1.4035777013576443, -1.2335200992327815, -1.1104252067708802, -1.0729293729304925, -0.9614589562411084, -0.8605589872168959, -0.9449568847777248, -0.9701971251393977, -0.9346295933822049, -0.9268888019901076, -0.7909766832562504, -0.8215956887587152, -0.8970344508089014, -0.9938742884075835, -1.123538857395495, -1.1890673315496838, -1.3036379219601426, -1.4211994968680393, -1.445504254771874, -1.3875243025505242, -1.4222190811545175, -1.509987864760724, -1.4460731906874233, -1.3739175483095805, -1.3750188909445296, -1.4206059785874139, -1.256102997436364, -1.1089307605393341, -1.0505591782148165, -1.0349728105382923, -1.057671231635621, -1.0583763602315

In [15]:
fig.update_layout(updatemenus=[dict(type='buttons',
                                  showactive=False,
                                  y=0.1,
                                  x=1.05,
                                  xanchor='left',
                                  yanchor='top',
                                  #pad=dict(t=1),
                                  buttons=[dict(label='Play',
                                                method='animate',
                                                args=[None, dict(frame=dict(duration=50, redraw=True), 
                                                                 transition_duration=0,
                                                                 fromcurrent=True,
                                                                 mode='immediate'
                                                                 )])
                                          ]
                                  )])   
fig.update(frames=frames);

In [16]:
import chart_studio.plotly as py
fig.write_html('yaw-roll-motion.html', auto_open=True)